En esta tarea aplicarás lo visto en clase para implementar **Leave-One-Out Cross Validation** para seleccionar el mejor hyperparámetro de una lista de datos y validadremos nuestra decisión considerando **k-fold Cross Validation**.

![Amsterdam](https://www.iamsterdam.com/media/canals-and-cityscapes/l-c-dentists-nc.jpg?h=328&w=580&as=false&iar=true)

Supongamos te quieres ir de viaje a Amsterdam y quedarte en algún lugar usando Airbnb. Para estimar cuanto te costaría el viaje decides usar un modelo lineal para saber cuanto afecta el precio dada una lista de *features*. Para tu suerte, encuentras una [base de datos](http://tomslee.net/airbnb-data-collection-get-the-data) con justo lo que necesitas. Evidentemente, la mejor manera de tomar esta decisión, para un(a) ñoño(a) como tú es usando machine learning.

Iniciamos nuestro análisis cargando las funciones que necesitarás para completar la tarea.

In [1]:
from numpy import argmin
from math import sqrt
from statistics import mean, stdev
from cv_utls import (load_airbnb,
                     train_cv_test_split,
                     get_feature_targets,
                     plot_cost,
                     estimate_theta,
                     make_train_fold,
                     l2_norm,
                     create_k_folds) 

Empezamos cargando la base de datos y limpiandola. Correr la siguiente línea baja los datos, los carga y nos regresa un tuple donde el primer elemento es la base de datos y, la segunda, el nombre de las variables con las cuáles predeciremos el costo.

Al cargar todos los datos, deberías ver un mapa con los precios para cada punto a considerar en nuestra base de datos.

In [ ]:
airbnb, airbnb_features = load_airbnb()

Al correr la siguiente línea de codigo veremos los *features* de nuestro modelo. Una cosa importante a notar son las variables `neighborhood_xxx` y `room_type_xxx`. Este es un ejemplo de *One-Hot-Encoding*.

Al tener una columna de variables categóricas, digámos:

|obs| feat |
|--|-----| 
|o1|A |
|o2|B |
|o3|C |
|o4|C |
|o5|B |

Podemos modificar esta columna para poder ingresarla a nuestro modelo de machine learning creando 3 nuevos *features*, uno para cada cada categoría. Evidentemente, una observación solo tendrá una clase asignada, es decir, un solo uno seguido de una fila de ceros. En nuestro ejemplo anterior, esto quedaría como sigue:

|obs|feat_A|feat_B|feat_C|
|--|-----|-----|-----|
|o1|1|0|0|
|o2|0|1|0|
|o3|0|0|1|
|o4|0|0|1|
|o5|0|1|0|

De esta manera, ya podemos entrenar nuestro modelo usando variables categóricas.

In [ ]:
airbnb_features

La función `train_cv_test_split` separa la base de datos en 3 componentes: train, cv y test.

In [ ]:
train, cv, test = train_cv_test_split(airbnb)
features_targets = get_feature_targets(train, cv, test)
X_train, X_cv, X_test = features_targets["features"]
y_train, y_cv, y_test = features_targets["targets"]

<h2 style="color:crimson"> Ejercicio </h2>
Implementa *Leave-One-Out Cross Validation*.

Empecemos por elegir el mejor hyperparámetro para nuestro modelo. Tu trabajo es completar la función `loocv` considerando la metodología vista en clase para este tipo de CV.

Para completar la función deberás usar las funciones
* `estimate_theta(X, y, lmbda)`: esta fución toma una serie de *features* $X$, *targets* $y$ y lmbda como factor de regularización. Date cuenta que ya tienes $X$, y $y$. Tu tarea es elegir la adecuada entre las variables que cargamos en la celda de arriba.
* `l2_norm(theta, X, y)`: Calcula el error considerando una norma en $L_2$ como vista en clase. Cuando cálcules cada Ji, por esta ocasión, divide el resultado de `l2_norm` por `nexamples` para normalizar el resultado. (El mínimo no se ve afectado por esta transformación)

**Nota**, Al igual que con loops, una función en python se declará con dos puntos después de los argumentos y se dejan cuatro espacios para toda linea subsecuente que pertencezca a la función. No te preocupes, veremos los detalles de la función en la clase de Python.

In [2]:
def loocv(lambdas, X_train, X_cv, y_train, y_cv):
    """
    Función para estimar el mejor hyperparámetro
    de una regressión lineal con penalización en L2
    (ridge regression) usando el método
    Leave-One-Out Cross Validation
    
    Parametros
    ----------
    lambdas: list
        Lista de posibles hyperparámetros que mejor
        generalizen X_cv
    """
    nexamples = X_cv.shape[1]
    ### Completar Código ##
    # Inicializa las variables 'Jcv' y 'thetas' como listas
    # vacías (aquí guardaremos las variables para tomar las decisiones)
    Jcv = None
    thetas = None
    # Itera sobre cada lambda_i dentro de la lista de lambdas, recuerda
    # usar las funciones estimate_theta y l2_norm y guardar tus costos en
    # Jcv y thetas
    
    return Jcv, thetas

In [ ]:
lambdas = [l/100 for l in range(15)]
Js, thetas = loocv(lambdas, X_train, X_cv, y_train, y_cv)
# Grafica la función objetivo (J) como función
# del hyperparámetro lambda
plot_cost(lambdas, Js)


icv = argmin(Js)
theta_cv = thetas[icv]
print("El índice óptimo es", icv)
print("Lambda óptima es", lambdas[icv])
print("J(theta) óptima es {:,.2f}".format(Js[icv]))

**Resultado esperado**
```
El índice óptimo es 11
Lambda óptima es 0.11
J(theta) óptima es 2,282.70
```

<h2 style="color:crimson"> Ejercicio </h2>
Corre la siguiente linea de código e interpreta tus resultados. Considera lo siguiente

1) ¿qué te dice cada uno de los coeficientes óptimos respecto a elegir un lugar para quedarse en Amsterdam?

In [ ]:
for theta, name in zip(theta_cv.ravel(), airbnb_features):
    print("{t:12,.2f} | {n}".format(t=theta, n=name))

<h2 style="color:crimson"> Ejercicio </h2>
Implementa *K-fold Cross Validation*.

Una vez elegido un modelo, nuestro tabajo ahora es saber que tan confiables son los parámetros encontrados.
Completa la función `kfoldcv` considerando la metodología en clase para este tipo de CV.

Recuerda que, para k-fold cv, ya asumimos una $\lambda$ y queremos probar su confiablidad probandola sobre cada base de datos. Para completar esta tarea tienes las funciones

* `create_k_folds(data, nfolds)` toma una base de datos a entrenar (*train* en este caso) y nos crea una lista de tuples nfolds pares con (X, y) valores para entrenar
* `make_rtrain_fold(folds, k)` toma una lista de tuples con (X, y) y un índice $k$, nos regresa una sola base de datos **sin** considerar el $k$-ésimo índice.

In [ ]:
def kfoldcv(lmbda, data, nfolds):
    """
    Función para estimar el mejor hyperparámetro
    de una regressión lineal con penalización en L2
    (ridge regression) usando el método de K-fold CV
    """
    # Crea 'nfolds' usando la función create_k_folds
    folds = create_k_folds(data, nfolds)
    Js, thetas = [], []
    for k in range(nfolds):
        # Obtén el el 'training dataset' omitiendo el k-ésimo
        # elemento en folds (considera la función make_train_fold)
        X_train, y_train = None
        # Obtén el k-ésimo fold dentro de folds
        X_cv, y_cv = None
        nexamples = X_cv.shape[1]
        # Estima theta^* considerando el hyperparámetros lmbda
        theta_i = None
        # Estima el error J. Recuerda dividir el costo entre
        # 'examples' para una interpretación respecto a la media.
        # De igual manera, calcula la raíz cuadrada de de Ji
        Ji = None
        Js.append(Ji)
        # Agregamos cada parámetro estimado a la lista
        # de estimaciones totales
        thetas.append(theta_i.ravel().tolist())
    
    return Js, thetas

In [ ]:
Jfolds, thetas = kfoldcv(0.11, train, 100)

# Calcula el el promedio de los errores estándard (Jfolds)
# usando la función 'mean'. Guardala dentro de la
# variable 'meanerr'
meanerr = None

# Calcula la desviación estándard de los errores estándard (Jfolds)
# usando la función 'stdev'. Guárdala dentro de la variable
# stdevs
stdevs = None

print("La média los errores estándard fueron {:,.2f}".format(meanerr))
print("La desviación promedio de los errores estándard fueron {:,.2f}".format(stdevs))

**Resultado esperado**

```
La média los errores estándard fueron 19.47
La desviación promedio de los errores estándard fueron 26.05
```

<h2 style="color:crimson">Ejercicio*</h2>
Considerando la lista `thetas` (recordemos que esta es una lista de listas con la estimación de cada parámetro), cálcula la `theta_i` promedio y su desviación estándard para toda `i`. Seguido de esto, presenta un intervalo de confianza a una desviación estándard para cada parámetro.

¿Qué tan confiables son los resultados? Argumenta tu respuesta

**Resultado Ejemplo**

```
   135.14 ± 959.05: bias
    14.98 ±  33.02: accommodates
    14.35 ±   9.84: bedrooms
    -3.55 ±   9.37: minstay
     5.58 ±  20.29: overall_satisfaction
    -5.37 ±  13.41: reviews
   -21.73 ± 122.90: neighborhood_BijlmerCentrum
   -14.77 ±  84.64: neighborhood_BijlmerOost
    -4.58 ±  87.45: neighborhood_BosenLommer
     7.13 ± 103.25: neighborhood_Buitenveldert_Zuidas
    51.08 ± 120.96: neighborhood_CentrumOost
    59.71 ± 109.05: neighborhood_CentrumWest
     2.35 ± 109.63: neighborhood_DeAker_NieuwSloten
    24.98 ± 131.17: neighborhood_DeBaarsjes_OudWest
    27.92 ± 128.60: neighborhood_DePijp_Rivierenbuurt
   -11.72 ± 149.29: neighborhood_Gaasperdam_Driemond
    -4.19 ±  85.82: neighborhood_Geuzenveld_Slotermeer
    -3.51 ±  81.30: neighborhood_Ijburg_EilandZeeburg
    27.80 ±  82.39: neighborhood_Noord-West_Noord-Midden
   -30.20 ±  89.31: neighborhood_NoordOost
   -18.75 ±  96.13: neighborhood_NoordWest
     1.16 ±  76.74: neighborhood_OostelijkHavengebied_IndischeBuurt
   -13.80 ±  88.04: neighborhood_Osdorp
    -4.06 ±  92.21: neighborhood_OudNoord
     3.59 ±  60.84: neighborhood_OudOost
   -10.87 ±  62.11: neighborhood_Slotervaart
     5.05 ±  76.97: neighborhood_Watergraafsmeer
     6.30 ±  74.24: neighborhood_Westerpark
    13.97 ± 110.08: neighborhood_Westpoort
   -49.31 ± 771.05: room_type_Entire_homeapt
   -80.99 ± 676.84: room_type_Private_room
  -114.85 ± 565.26: room_type_Shared_room
```

<h2 style="color:crimson">Ejercicio</h2>
Como ejercicio final, calcula el costo $J(\theta|X^{test}, y^{test})$ considerando el test set. Recuerda dividir tu costo enttre el número de ejemplos guardado en la variable `ntest`

In [ ]:
ntest = X_test.shape[1]
cost_test = None
cost_test

**Resultado esperado**
```
2271.8588520664762
```